<a href="https://colab.research.google.com/github/sugitora/-AutoSendGmail/blob/master/20240501_An_Introduction_to_the_Global_YouTube_Statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'global-youtube-statistics-2023:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3566603%2F6211042%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240501%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240501T100947Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4d792371478b8f6b6110090ea9bb9116b214049084dfd96fe813c0e238e726c4f4aa45b0d455b6546c680c44ec38b2a9b71b195d4aaffcacdcbb6220d793f84de2b28b0e939ffed096f901df768a6b7b3a0150c7b1dba06b6323fe16e56e041c1184f7397a57ea9e65e222f0db6834ea77c49529ea8e99c3e1ecbd35d3d21707510880ec8175b42ac54079cd982130f59f24360cdaadf5c5b9a4aa8739d425e5ee463126a9faa6a44a6eae16f4bcf313e34e063391dc266d4486a924fb1941a58a7ebc79562f614cf52c5df807492544d603a4c0d6ba4214a5f00925b36018f826510c61584904f49da5c909effe660452b63029ace29bebedb71ab6d34faa53'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 61533 bytes downloaded
Downloaded and uncompressed: global-youtube-statistics-2023
Data source import complete.


<center> <h1> An Introduction to the Dataset </h1>
    <h3>Global YouTube Statistics 2023</h3><br>

<center>This is a sample notebook providing an overview of the meticulously curated dataset that unveils the statistics of the most subscribed YouTube channels. A collection of YouTube giants, this dataset offers a perfect avenue to analyze and gain valuable insights from the luminaries of the platform. With comprehensive details on top creators' subscriber counts, video views, upload frequency, country of origin, earnings, and more, this treasure trove of information is a must-explore for aspiring content creators, data enthusiasts, and anyone intrigued by the ever-evolving online content landscape. Immerse yourself in the world of YouTube success and unlock a wealth of knowledge with this extraordinary dataset..

<center> <h1> データセットへの序章 </h1>
    <h3>2023年のグローバルYouTube統計</h3><br>
<center>これは、最も登録者数の多いYouTubeチャンネルの統計を明らかにする、入念にキュレーションされたデータセットの概要を提供するサンプルノートブックである。
 YouTubeの巨人たちの集まりであるこのデータセットは、プラットフォームの先駆者たちから貴重な洞察を分析し、得るための完璧な機会を提供する。
 トップクリエイターの登録者数、動画視聴回数、アップロード頻度、出身国、収益などの包括的な詳細が含まれているため、この情報の宝庫は、野心的なコンテンツクリエイター、データ愛好家、そして常に進化するオンラインコンテンツの風景に魅了される全ての人にとって、探求すべき必須アイテムである。  
 YouTubeの成功の世界に没頭し、この特別なデータセットで知識の宝庫を解き放とう。

In [25]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import warnings


warnings.filterwarnings("ignore")

# World

In [26]:
df = pd.read_csv("/kaggle/input/global-youtube-statistics-2023/Global YouTube Statistics.csv", encoding="latin-1")

In [27]:
# import pandas as pd

# try:
#     # cp932エンコーディングを試みる
#     df = pd.read_csv("/kaggle/input/global-youtube-statistics-2023/Global YouTube Statistics.csv", encoding="cp932")
# except UnicodeDecodeError:
#     # shift_jisエンコーディングを試みる
#     df = pd.read_csv("/kaggle/input/global-youtube-statistics-2023/Global YouTube Statistics.csv", encoding="shift_jis")

# # 日本のチャンネルのみを抽出
# japan_channels = df[df['Country'] == 'Japan']

# # 抽出したデータの先頭を表示
# print(japan_channels.head())


In [28]:
df.describe()

,rank,subscribers,video views,uploads,video_views_rank,country_rank,channel_type_rank,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,...,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
count,995.00000,9.950000e+02,9.950000e+02,995.000000,9.940000e+02,879.000000,962.000000,9.390000e+02,995.000000,9.950000e+02,...,9.950000e+02,6.580000e+02,990.000000,990.000000,872.000000,8.720000e+02,872.000000,8.720000e+02,872.000000,872.000000
mean,498.00000,2.298241e+07,1.103954e+10,9187.125628,5.542489e+05,386.053470,745.719335,1.756103e+08,36886.148281,5.898078e+05,...,7.081814e+06,3.490791e+05,2012.630303,15.746465,63.627752,4.303873e+08,9.279278,2.242150e+08,26.632783,-14.128146
std,287.37606,1.752611e+07,1.411084e+10,34151.352254,1.362782e+06,1232.244746,1944.386561,4.163782e+08,71858.724092,1.148622e+06,...,1.379704e+07,6.143554e+05,4.512503,8.777520,26.106893,4.727947e+08,4.888354,1.546874e+08,20.560533,84.760809
min,1.00000,1.230000e+07,0.000000e+00,0.000000,1.000000e+00,1.000000,1.000000,1.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,1.000000e+00,1970.000000,1.000000,7.600000,2.025060e+05,0.750000,3.558800e+04,-38.416097,-172.104629
25%,249.50000,1.450000e+07,4.288145e+09,194.500000,3.230000e+02,11.000000,27.000000,2.013750e+07,2700.000000,4.350000e+04,...,5.217500e+05,1.000000e+05,2009.000000,8.000000,36.300000,8.335541e+07,5.270000,5.590832e+07,20.593684,-95.712891
50%,498.00000,1.770000e+07,7.760820e+09,729.000000,9.155000e+02,51.000000,65.500000,6.408500e+07,13300.000000,2.127000e+05,...,2.600000e+06,2.000000e+05,2013.000000,16.000000,68.000000,3.282395e+08,9.365000,2.706630e+08,37.090240,-51.925280
75%,746.50000,2.460000e+07,1.355470e+10,2667.500000,3.584500e+03,123.000000,139.750000,1.688265e+08,37900.000000,6.068000e+05,...,7.300000e+06,4.000000e+05,2016.000000,23.000000,88.200000,3.282395e+08,14.700000,2.706630e+08,37.090240,78.962880
max,995.00000,2.450000e+08,2.280000e+11,301308.000000,4.057944e+06,7741.000000,7741.000000,6.589000e+09,850900.000000,1.360000e+07,...,1.634000e+08,8.000000e+06,2022.000000,31.000000,113.100000,1.397715e+09,14.720000,8.429340e+08,61.924110,138.252924


もちろんです。以下に「である調」で表現を修正しました。

### 基本統計

- **count**: 各列の非欠損値の数である。
- **mean**: 平均値である。
- **std**: 標準偏差である。
- **min**: 最小値である。
- **25%**: 第一四分位数（25パーセンタイル）である。
- **50%**: 中央値（50パーセンタイル）である。
- **75%**: 第三四分位数（75パーセンタイル）である。
- **max**: 最大値である。

### 各列の具体的な解析

- **rank**: YouTubeチャンネルのランキングで、995チャンネルのデータが存在する。ランキングの平均は498である。
- **subscribers**: 登録者数で、平均は約2298万人、最小は1230万人、最大は2億4500万人である。
- **video views**: 動画視聴回数で、平均は約110億回、最小は0回、最大は2280億回である。
- **uploads**: アップロードされた動画の数で、平均は9187本、最小は0本、最大は301,308本である。
- **video_views_rank**: 動画視聴数に基づくランキングで、最小は1、最大は4057944である。
- **country_rank**, **channel_type_rank**: 国別、チャンネルタイプ別のランキングである。
- **video_views_for_the_last_30_days**: 直近30日間の動画視聴回数で、平均は約1.76億回、最小は1回、最大は65.89億回である。
- **lowest_monthly_earnings**, **highest_monthly_earnings**: 月間最低および最高収益で、平均では最低収益が約36,886ドル、最高収益が約589,807ドルである。
- **highest_yearly_earnings**: 年間最高収益で、平均は約708万ドル、最大は1億6340万ドルである。
- **subscribers_for_last_30_days**: 直近30日間の登録者数増加で、平均は約34.9万人、最大は800万人である。
- **created_year**: チャンネル作成年で、最古は1970年、最新は2022年である。
- **created_date**, **Gross tertiary education enrollment (%)**, **Population**, **Unemployment rate**, **Urban_population**, **Latitude**, **Longitude**: これらの列は国や地域に関する統計情報を示しており、人口統計や経済状況など、YouTubeチャンネルのコンテキストに関連するデータを提供するものである。

これらの統計は、YouTubeチャンネルの規模、成長、およびその他の特性を理解するのに役立つ貴重な洞察を提供するものである。特に、視聴回数や登録者数の増加に関するデータは、チャンネルのパフォーマンスを評価するのに非常に重要である。

In [29]:
df.head(5)

,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,...,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
0,1,T-Series,245000000,2.280000e+11,Music,T-Series,20082,India,IN,Music,...,2000000.0,2006.0,Mar,13.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880
1,2,YouTube Movies,170000000,0.000000e+00,Film & Animation,youtubemovies,1,United States,US,Games,...,NaN,2006.0,Mar,5.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
2,3,MrBeast,166000000,2.836884e+10,Entertainment,MrBeast,741,United States,US,Entertainment,...,8000000.0,2012.0,Feb,20.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
3,4,Cocomelon - Nursery Rhymes,162000000,1.640000e+11,Education,Cocomelon - Nursery Rhymes,966,United States,US,Education,...,1000000.0,2006.0,Sep,1.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
4,5,SET India,159000000,1.480000e+11,Shows,SET India,116536,India,IN,Entertainment,...,1000000.0,2006.0,Sep,20.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880


このデータは、YouTubeの最も登録者数の多いトップ5のチャンネルに関する統計情報をまとめたものである。
以下は各チャンネルの概要である：

1. **T-Series（インド）**
   - **登録者数**: 245,000,000人
   - **動画視聴回数**: 228,000,000,000回
   - **カテゴリ**: 音楽
   - **アップロード数**: 20,082本
   - **直近30日の視聴回数**: 2,258,000,000回
   - **直近30日の登録者増加数**: 2,000,000人

2. **YouTube Movies（アメリカ合衆国）**
   - **登録者数**: 170,000,000人
   - **動画視聴回数**: 0回
   - **カテゴリ**: 映画・アニメーション
   - **アップロード数**: 1本
   - **直近30日の視聴回数**: 12回
   - **直近30日の登録者増加数**: 記載なし（NaN）

3. **MrBeast（アメリカ合衆国）**
   - **登録者数**: 166,000,000人
   - **動画視聴回数**: 28,368,841,870回
   - **カテゴリ**: エンターテイメント
   - **アップロード数**: 741本
   - **直近30日の視聴回数**: 1,348,000,000回
   - **直近30日の登録者増加数**: 8,000,000人

4. **Cocomelon - Nursery Rhymes（アメリカ合衆国）**
   - **登録者数**: 162,000,000人
   - **動画視聴回数**: 164,000,000,000回
   - **カテゴリ**: 教育
   - **アップロード数**: 966本
   - **直近30日の視聴回数**: 1,975,000,000回
   - **直近30日の登録者増加数**: 1,000,000人

5. **SET India（インド）**
   - **登録者数**: 159,000,000人
   - **動画視聴回数**: 148,000,000,000回
   - **カテゴリ**: ショー
   - **アップロード数**: 116,536本
   - **直近30日の視聴回数**: 1,824,000,000回
   - **直近30日の登録者増加数**: 1,000,000人

YouTube Moviesについての特異点の説明は以下である。YouTube Moviesは、YouTubeが提供する映画販売・レンタルサービスを展開するチャンネルである。このチャンネルは主に映画のレンタルや購入を目的としており、そのためアップロード数が非常に少なく、視聴回数も非常に限定的である。通常のYouTubeチャンネルとは異なり、エンゲージメントや視聴回数を増やすことが主目的ではないため、登録者数に比べて動画視聴回数やアップロード数が非常に少ない状況が生じている。

In [ ]:
df_cleaned = df.replace('nan', pd.NA)  # Replace 'nan' with pandas' NA for numeric columns
numeric_columns = ['rank', 'subscribers', 'video views', 'video_views_for_the_last_30_days',
                   'lowest_monthly_earnings', 'highest_monthly_earnings', 'lowest_yearly_earnings',
                   'highest_yearly_earnings', 'subscribers_for_last_30_days', 'created_year', 'Unemployment rate']
for col in numeric_columns:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

df_cleaned = df_cleaned.dropna()

top_10_subscribers = df_cleaned.nlargest(10, 'subscribers')
top_10_subscribers

,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,...,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
0,1,T-Series,245000000,2.280000e+11,Music,T-Series,20082,India,IN,Music,...,2000000.0,2006.0,Mar,13.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880
2,3,MrBeast,166000000,2.836884e+10,Entertainment,MrBeast,741,United States,US,Entertainment,...,8000000.0,2012.0,Feb,20.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
3,4,Cocomelon - Nursery Rhymes,162000000,1.640000e+11,Education,Cocomelon - Nursery Rhymes,966,United States,US,Education,...,1000000.0,2006.0,Sep,1.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
4,5,SET India,159000000,1.480000e+11,Shows,SET India,116536,India,IN,Entertainment,...,1000000.0,2006.0,Sep,20.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880
8,9,Like Nastya,106000000,9.047906e+10,People & Blogs,Like Nastya Vlog,493,Russia,RU,People,...,100000.0,2016.0,Jan,14.0,81.9,1.443735e+08,4.59,107683889.0,61.524010,105.318756
9,10,Vlad and Niki,98900000,7.718017e+10,Entertainment,Vlad and Niki,574,United States,US,Entertainment,...,600000.0,2018.0,Apr,23.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
10,11,Zee Music Company,96700000,5.785629e+10,Music,Zee Music Company,8548,India,IN,Music,...,1100000.0,2014.0,Mar,12.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880
11,12,WWE,96000000,7.742847e+10,Sports,WWE,70127,United States,US,Sports,...,600000.0,2007.0,May,11.0,88.2,3.282395e+08,14.70,270663028.0,37.090240,-95.712891
13,14,BLACKPINK,89800000,3.214460e+10,People & Blogs,BLACKPINK,543,South Korea,KR,Music,...,700000.0,2016.0,Jun,29.0,94.3,5.170910e+07,4.15,42106719.0,35.907757,127.766922
15,16,Sony SAB,83000000,1.010000e+11,Shows,Sony SAB,71270,India,IN,Entertainment,...,1100000.0,2007.0,Aug,4.0,28.1,1.366418e+09,5.36,471031528.0,20.593684,78.962880


提供されたデータは、世界中のさまざまなYouTubeチャンネルに関する詳細な情報を含む表である。以下はそれぞれのチャンネルに関する主要な情報である：

1. **T-Series (India)**
   - **ランク**: 1
   - **登録者数**: 245,000,000人
   - **動画視聴回数**: 228,000,000,000回
   - **カテゴリ**: 音楽
   - **アップロード数**: 20,082本
   - **チャンネルタイプ**: 音楽
   - **直近30日の登録者増加数**: 2,000,000人
   - **チャンネル作成年**: 2006年
   - **チャンネル作成月**: 3月
   - **人口**: 1,366,418,000人
   - **失業率**: 5.36%

2. **MrBeast (United States)**
   - **ランク**: 3
   - **登録者数**: 166,000,000人
   - **動画視聴回数**: 28,368,841,870回
   - **カテゴリ**: エンターテイメント
   - **アップロード数**: 741本
   - **チャンネルタイプ**: エンターテイメント
   - **直近30日の登録者増加数**: 8,000,000人
   - **チャンネル作成年**: 2012年
   - **チャンネル作成月**: 2月
   - **人口**: 328,239,500人
   - **失業率**: 14.70%

3. **Cocomelon - Nursery Rhymes (United States)**
   - **ランク**: 4
   - **登録者数**: 162,000,000人
   - **動画視聴回数**: 164,000,000,000回
   - **カテゴリ**: 教育
   - **アップロード数**: 966本
   - **チャンネルタイプ**: 教育
   - **直近30日の登録者増加数**: 1,000,000人
   - **チャンネル作成年**: 2006年
   - **チャンネル作成月**: 9月
   - **人口**: 328,239,500人
   - **失業率**: 14.70%

4. **SET India (India)**
   - **ランク**: 5
   - **登録者数**: 159,000,000人
   - **動画視聴回数**: 148,000,000,000回
   - **カテゴリ**: ショー
   - **アップロード数**: 116,536本
   - **チャンネルタイプ**: エンターテイメント
   - **直近30日の登録者増加数**: 1,000,000人
   - **チャンネル作成年**: 2006年
   - **チャンネル作成月**: 9月
   - **人口**: 1,366,418,000人
   - **失業率**: 5.36%

5. **Like Nastya (Russia)**
   - **ランク**: 9
   - **登録者数**: 106,000,000人
   - **動画視聴回数**: 90,479,060,027回
   - **カテゴリ**: 人々とブログ
   - **アップロード数**: 493本
   - **チャンネルタイプ**: 人々
   - **直近30日の登録者増加数**: 100,000人
   - **チャンネル作成年**: 2016年
   - **チャンネル作成月**: 1月


   - **人口**: 144,373,500人
   - **失業率**: 4.59%

これらのデータは、それぞれのチャンネルの人気度、市場の位置づけ、及び社会経済的背景に関する洞察を提供する。

In [ ]:
fig1 = px.bar(top_10_subscribers, x='subscribers', y='Youtuber', orientation='h', text='subscribers',
              color='subscribers', labels={'subscribers': 'Subscribers (in billions)'},
              color_continuous_scale='Viridis')
fig1.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5,
                    opacity=0.8, textposition='inside')
fig1.update_layout(title_text='Top 10 YouTube Channels by Subscribers', yaxis_title='YouTube Channel',
                   xaxis_title='Subscribers (in billions)', height=500)
fig1.show()

このグラフは、登録者数に基づいてトップ10のYouTubeチャンネルを水平バーで表示しているものである。各バーの長さはチャンネルの登録者数を示している。バーは「subscribers」軸に沿って配置され、右に長いほど多くの登録者数を持っていることを意味している。

- 最も登録者数が多いのは「T-Series」で、約2億4500万人である。
- 次に「MrBeast」で、登録者数は約1億6600万人である。
- 「Cocomelon - Nursery Rhymes」は約1億6200万人の登録者を持つ。
- 「SET India」は約1億5900万人の登録者を有している。
- その他のチャンネルも1億人を超える登録者数を持っていることが示されている。

グラフの色彩は「Viridis」カラースケールを使用しており、登録者数に応じて色が変化している。この視覚的表現は、各チャンネルの相対的な規模を一目で理解するのに役立っている。

グラフのタイトル「Top 10 YouTube Channels by Subscribers」は、このグラフが登録者数によるYouTubeチャンネルのトップ10を示していることを明確にしている。X軸のタイトル「Subscribers (in billions)」は、登録者数が何十億人単位で表示されていることを示しており、Y軸のタイトル「YouTube Channel」は、表示されているデータがYouTubeチャンネルに関するものであることを示している。

In [ ]:
fig1 = go.Figure(data=go.Scatter3d(
    x=df_cleaned['subscribers'],
    y=df_cleaned['video views'],
    z=df_cleaned['uploads'],
    mode='markers',
    marker=dict(
        size=8,
        color=df_cleaned['video_views_for_the_last_30_days'],
        colorscale='Viridis',
        opacity=0.7
    )
))

fig1.update_layout(title='YouTube Channels: Subscribers, Video Views, and Uploads',
                   scene=dict(xaxis_title='Subscribers', yaxis_title='Video Views', zaxis_title='Uploads'),
                   margin=dict(l=0, r=0, b=0, t=30))
fig1.show()

このグラフは、YouTubeチャンネルの登録者数、動画視聴回数、およびアップロード数を3次元の散布図で示しているものである。このグラフでは、各点がYouTubeチャンネルを表しており、x軸は「登録者数」、y軸は「動画視聴回数」、z軸は「アップロード数」に対応している。

- **登録者数**: x軸に位置し、左から右に増加している。最大で数十億の登録者数を示している。
- **動画視聴回数**: y軸に位置し、手前から奥に増加している。最大で数百億回の視聴がある。
- **アップロード数**: z軸に位置し、下から上に増加している。最大で数十万本のアップロードがある。

散布図のマーカーは、直近30日間の動画視聴回数に基づいて色分けされている。これにより、チャンネルの最近の活動性や人気を視覚的に捉えることが可能である。色の濃淡は「Viridis」カラースケールを使用し、視聴回数が多いほど色が濃くなっている。

このグラフの設計は、登録者数、動画視聴回数、アップロード数の三つの主要な指標を同時に考慮することで、YouTubeチャンネルの規模と活動の広がりを多角的に理解するためのものである。タイトル「YouTube Channels: Subscribers, Video Views, and Uploads」は、このグラフがYouTubeチャンネルのこれらの側面を探るものであることを示している。

In [ ]:
#  Scatter plot for subscribers vs. video views with size and color encoding
fig3 = px.scatter(df_cleaned, x='subscribers', y='video views', size='video_views_for_the_last_30_days',
                  color='category', hover_name='Youtuber', hover_data=['Country'], size_max=40,
                  labels={'subscribers': 'Subscribers (in billions)', 'video views': 'Video Views (in billions)'},
                  color_discrete_sequence=px.colors.qualitative.Set2)
fig3.update_layout(title_text='Subscribers vs. Video Views by Category', xaxis_type='log', yaxis_type='log')
fig3.show()

このグラフは、YouTubeチャンネルの登録者数と動画視聴回数をカテゴリ別に示している散布図である。x軸は「登録者数（数十億単位）」を、y軸は「動画視聴回数（数十億単位）」を示しており、両軸ともに対数スケールで表示されている。

- 各点の大きさは、直近30日間の動画視聴回数に基づいており、大きいほど視聴回数が多いことを意味している。
- 色はカテゴリに基づいており、異なるカテゴリが色分けされている。これにより、特定のカテゴリが登録者数や動画視聴回数においてどのような位置を占めているかが一目で分かる。

例えば、大きな緑色の点は「教育」カテゴリを示しており、登録者数が少ないにも関わらず動画視聴回数が非常に多いことがわかる。一方、オレンジ色の点は「音楽」カテゴリで、登録者数も動画視聴回数も非常に多いことを示している。

このグラフのタイトル「Subscribers vs. Video Views by Category」は、このグラフがカテゴリ別の登録者数と動画視聴回数の関係を示していることを明確にしている。これにより、異なるカテゴリのYouTubeチャンネルがどのようにパフォーマンスしているかを評価することが可能である。

In [ ]:
# Sunburst chart for the distribution of channel types
fig2 = px.sunburst(df_cleaned, path=['channel_type'], color_discrete_sequence=px.colors.qualitative.Set3)
fig2.update_layout(title_text='Distribution of YouTube Channel Types', height=500)
fig2.show()

提供されたグラフは、YouTubeのさまざまなチャンネルタイプの分布を示す円グラフである。このグラフは、YouTube上のチャンネルがどのカテゴリに属しているかの割合を色分けして表示している。

- **「エンターテイメント」** が最も大きなセグメントであり、多くのYouTubeチャンネルがこのカテゴリに属していることが示されている。
- **「音楽」** もかなり大きな割合を占めており、これはYouTubeが音楽ビデオやアーティストのプロモーションに広く利用されていることを反映している。
- その他のカテゴリとしては、**「ゲーム」**、**「人々」**（おそらく人物やブロガーを対象としたカテゴリ）、**「教育」** などがある。

各セグメントの色は異なっており、視覚的にどのカテゴリがどれだけの割合を占めているかを容易に理解できるようになっている。タイトル「Distribution of YouTube Channel Types」は、このグラフがYouTubeチャンネルのカテゴリ別分布を示していることを明確にしている。この情報は、コンテンツ制作者やマーケターがどのカテゴリが飽和しているか、またはどのカテゴリに機会があるかを理解するのに役立つ。

In [32]:
# # Scatter plot for subscribers vs. video views
# fig3 = px.scatter(df_cleaned, x='subscribers', y='video views', size='video_views_for_the_last_30_days',
#                   color='category', hover_name='Youtuber', hover_data=['Country'], size_max=40,
#                   labels={'subscribers': 'Subscribers (in billions)', 'video views': 'Video Views (in billions)'},
#                   color_discrete_sequence=px.colors.qualitative.Set2)
# fig3.update_layout(title_text='Subscribers vs. Video Views by Category', xaxis_type='log', yaxis_type='log')
# fig3.show()


In [ ]:
earnings_columns = ['lowest_monthly_earnings', 'highest_monthly_earnings', 'lowest_yearly_earnings', 'highest_yearly_earnings']
earnings_df = df_cleaned[earnings_columns]
fig4 = px.box(earnings_df, title='Distribution of Earnings',
              labels={'variable': 'Earnings', 'value': 'Earnings (in USD)'})
fig4.update_layout(height=500)
fig4.show()


提供されたグラフは、YouTubeチャンネルの収益の分布を示す箱ひげ図である。このグラフには、「最低月収」、「最高月収」、「最低年収」、「最高年収」の4つのカテゴリが含まれている。

- **最低月収**：このカテゴリの収益は比較的低く、大部分が0に近い値である。ただし、いくつかの外れ値も見られる。
- **最高月収**：このカテゴリの収益は少し高いが、最低月収よりも分布が広がっている。特に上位の外れ値が多く、収益の上限がかなり高いことが示されている。
- **最低年収**：このカテゴリの収益は最低月収と似ており、ほとんどが低い値であるが、外れ値は見られない。
- **最高年収**：このカテゴリでは、非常に高い収益が得られているチャンネルがいくつか存在する。最高年収の分布は、非常に高額な収益を示す点で顕著であり、最高値は100百万ドル以上にも達している。

グラフのタイトル「Distribution of Earnings」は、これらの収益データがYouTubeチャンネルの稼ぎ方の分布を示していることを明確にしている。この情報は、YouTubeチャンネルの収益潜在力を理解するのに役立つ。各カテゴリの収益分布を把握することで、YouTubeでの収益化の可能性や変動の大きさが理解される。

# Japan

In [ ]:
# データフレーム 'df' から日本のチャンネルのみを抽出
japan_channels = df[df['Country'] == 'Japan']


In [ ]:
# 抽出したデータの統計情報を表示
japan_channels.describe()

,rank,subscribers,video views,uploads,video_views_rank,country_rank,channel_type_rank,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,...,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
count,5.000000,5.000000e+00,5.000000e+00,5.000000,5.000000e+00,5.000000,5.000000,5.000000e+00,5.000000,5.000000e+00,...,5.000000e+00,4.000000e+00,5.000000,5.000000,5.0,5.0,5.00,5.0,5.000000,5.000000
mean,387.800000,3.782000e+07,1.713640e+10,3771.200000,6.937756e+05,703.800000,689.600000,3.509321e+08,87740.124000,1.391342e+06,...,1.684002e+07,7.000022e+05,2015.400000,19.600000,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
std,303.935355,4.119068e+07,7.583645e+09,2716.606468,1.550967e+06,1567.596153,1443.384668,3.508737e+08,87723.775572,1.396907e+06,...,1.681226e+07,4.830415e+05,4.878524,6.618157,0.0,0.0,0.00,0.0,0.000000,0.000000
min,8.000000,1.490000e+07,8.615619e+09,93.000000,4.400000e+01,1.000000,4.000000,2.477000e+03,0.620000,1.000000e+01,...,1.190000e+02,9.000000e+00,2010.000000,13.000000,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
25%,216.000000,1.500000e+07,1.356285e+10,1788.000000,1.440000e+02,2.000000,29.000000,3.918400e+07,9800.000000,1.567000e+05,...,1.900000e+06,6.000022e+05,2012.000000,14.000000,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
50%,321.000000,2.200000e+07,1.609753e+10,4716.000000,1.840000e+02,3.000000,56.000000,2.823720e+08,70600.000000,1.100000e+06,...,1.360000e+07,8.500000e+05,2014.000000,19.000000,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
75%,691.000000,2.620000e+07,1.834797e+10,5985.000000,2.770000e+02,5.000000,88.000000,7.112540e+08,177800.000000,2.800000e+06,...,3.410000e+07,9.500000e+05,2020.000000,23.000000,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
max,703.000000,1.110000e+08,2.905804e+10,6274.000000,3.468229e+06,3508.000000,3271.000000,7.218480e+08,180500.000000,2.900000e+06,...,3.460000e+07,1.100000e+06,2021.000000,29.000000,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924


提供されたデータは、YouTubeチャンネルに関するさまざまな統計指標の要約統計である。ここでの各列の意味は次の通りである：

- **count**: 各列のデータ数である。
- **mean**: 平均値である。
- **std**: 標準偏差である。
- **min**: 最小値である。
- **25%**: 第一四分位数である。
- **50%**: 中央値である。
- **75%**: 第三四分位数である。
- **max**: 最大値である。

### 各統計指標の具体的な内容

- **rank**: チャンネルのランキングである。
- **subscribers**: 登録者数である。
- **video views**: 動画の視聴回数である。
- **uploads**: アップロードされた動画の数である。
- **video_views_rank**: 動画視聴数に基づくランキングである。
- **country_rank**: 国別ランキングである。
- **channel_type_rank**: チャンネルタイプに基づくランキングである。
- **video_views_for_the_last_30_days**: 直近30日間の動画視聴回数である。
- **lowest_monthly_earnings**: 最低月間収益である。
- **highest_monthly_earnings**: 最高月間収益である。
- **highest_yearly_earnings**: 最高年間収益である。
- **subscribers_for_last_30_days**: 直近30日間の登録者数の増加である。
- **created_year**: チャンネルが作成された年である。
- **created_date**: チャンネルが作成された日である。
- **Gross tertiary education enrollment (%)**: 高等教育への総入学率である。
- **Population**: 人口である。
- **Unemployment rate**: 失業率である。
- **Urban_population**: 都市人口である。
- **Latitude**, **Longitude**: 緯度と経度である。

これらの統計は、分析対象のYouTubeチャンネルのパフォーマンスや人気度、経済的成功を把握するのに役立つ指標である。

In [ ]:
# 抽出したデータの先頭を表示
japan_channels.head()

,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,...,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
7,8,PewDiePie,111000000,2.905804e+10,Gaming,PewDiePie,4716,Japan,JP,Entertainment,...,NaN,2010.0,Apr,29.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
215,216,Junya.ï¿½ï¿½ï¿½ï¿½,26200000,1.609753e+10,Entertainment,Junya.ï¿½ï¿½ï¿½ï¿½,5985,Japan,JP,Entertainment,...,1100000.0,2020.0,Sep,14.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
320,321,Sagawa /ï¿½ï¿½ï¿½,22000000,1.834797e+10,Entertainment,Sagawa /ï¿½ï¿½ï¿½,6274,Japan,JP,Entertainment,...,800000.0,2021.0,Feb,13.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
690,691,ISSEI / ï¿½ï¿½ï¿½ï¿½,15000000,1.356285e+10,Comedy,ISSEI / ï¿½ï¿½ï¿½ï¿½,1788,Japan,JP,Comedy,...,900000.0,2014.0,Jul,23.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
702,703,Bayashi TV,14900000,8.615619e+09,Entertainment,ýýýýýýýýýýýýýýýýýýBAYASHITV,93,Japan,JP,Autos,...,9.0,2012.0,Nov,19.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924


このデータは、いくつかのYouTubeチャンネルに関する詳細情報を含んでいるが、文字化けが発生している部分が見受けられる。文字化けは、チャンネル名やタイトルの部分に特に顕著である。それでも、利用可能な情報から分析を進めることができる。以下は、各チャンネルに関する主要な情報である：

1. **PewDiePie (Japan)**
   - **ランク**: 8
   - **登録者数**: 111,000,000人
   - **動画視聴回数**: 29,058,044,447回
   - **カテゴリ**: ゲーム
   - **アップロード数**: 4,716本
   - **国別ランク**: 1位
   - **チャンネルタイプランク**: 4位
   - **直近30日の視聴回数**: 39,184,000回
   - **最低月収**: $9,800
   - **最高月収**: $156,700
   - **作成年**: 2010年

2. **Junya (Japan) [文字化け]**
   - **ランク**: 216
   - **登録者数**: 26,200,000人
   - **動画視聴回数**: 16,097,531,087回
   - **カテゴリ**: エンターテイメント
   - **アップロード数**: 5,985本
   - **国別ランク**: 2位
   - **チャンネルタイプランク**: 56位
   - **直近30日の視聴回数**: 721,848,000回
   - **最低月収**: $180,500
   - **最高月収**: $2,900,000
   - **作成年**: 2020年

3. **Sagawa (Japan) [文字化け]**
   - **ランク**: 321
   - **登録者数**: 22,000,000人
   - **動画視聴回数**: 18,347,969,186回
   - **カテゴリ**: エンターテイメント
   - **アップロード数**: 6,274本
   - **国別ランク**: 3位
   - **チャンネルタイプランク**: 88位
   - **直近30日の視聴回数**: 711,254,000回
   - **最低月収**: $177,800
   - **最高月収**: $2,800,000
   - **作成年**: 2021年

4. **ISSEI (Japan) [文字化け]**
   - **ランク**: 691
   - **登録者数**: 15,000,000人
   - **動画視聴回数**: 13,562,853,889回
   - **カテゴリ**: コメディ
   - **アップロード数**: 1,788本
   - **国別ランク**: 5位
   - **チャンネルタイプランク**: 29位
   - **直近30日の視聴回数**: 282,372,000回
   - **最低月収**: $70,600
   - **最高月収**: $1,100,000
   - **作成年**: 2014年

5. **Bayashi TV (Japan)**
   - **ランク**: 703
   - **登録者数**: 14,900,000人
   - **動画視聴回数**: 8,615,618,825回
   - **カテゴリ**: オートモーティブ
   - **アップロード数**: 93本
   - **国別ランク**: 3508位
   - **チャンネルタイプランク**:

In [42]:
# データフレーム 'df' から日本のチャンネルのみを抽出
japan_channels = df[df['Country'] == 'Japan']

In [43]:
japan_channels

,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,...,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
7,8,PewDiePie,111000000,2.905804e+10,Gaming,PewDiePie,4716,Japan,JP,Entertainment,...,NaN,2010.0,Apr,29.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
215,216,Junya.ï¿½ï¿½ï¿½ï¿½,26200000,1.609753e+10,Entertainment,Junya.ï¿½ï¿½ï¿½ï¿½,5985,Japan,JP,Entertainment,...,1100000.0,2020.0,Sep,14.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
320,321,Sagawa /ï¿½ï¿½ï¿½,22000000,1.834797e+10,Entertainment,Sagawa /ï¿½ï¿½ï¿½,6274,Japan,JP,Entertainment,...,800000.0,2021.0,Feb,13.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
690,691,ISSEI / ï¿½ï¿½ï¿½ï¿½,15000000,1.356285e+10,Comedy,ISSEI / ï¿½ï¿½ï¿½ï¿½,1788,Japan,JP,Comedy,...,900000.0,2014.0,Jul,23.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
702,703,Bayashi TV,14900000,8.615619e+09,Entertainment,ýýýýýýýýýýýýýýýýýýBAYASHITV,93,Japan,JP,Autos,...,9.0,2012.0,Nov,19.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924


In [39]:
# 'nan' を pandas の NA に置き換える
japan_channels = japan_channels.replace('nan', pd.NA)

# 数値列を指定し、数値型に変換
numeric_columns = ['rank', 'subscribers', 'video views', 'video_views_for_the_last_30_days',
                   'lowest_monthly_earnings', 'highest_monthly_earnings', 'lowest_yearly_earnings',
                   'highest_yearly_earnings', 'subscribers_for_last_30_days', 'created_year', 'Unemployment rate']
for col in numeric_columns:
    japan_channels[col] = pd.to_numeric(japan_channels[col], errors='coerce')

# 欠損値を含む行を削除
japan_channels = japan_channels.dropna()


In [41]:
japan_channels

,rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,...,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
215,216,Junya.ï¿½ï¿½ï¿½ï¿½,26200000,1.609753e+10,Entertainment,Junya.ï¿½ï¿½ï¿½ï¿½,5985,Japan,JP,Entertainment,...,1100000.0,2020.0,Sep,14.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
320,321,Sagawa /ï¿½ï¿½ï¿½,22000000,1.834797e+10,Entertainment,Sagawa /ï¿½ï¿½ï¿½,6274,Japan,JP,Entertainment,...,800000.0,2021.0,Feb,13.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
690,691,ISSEI / ï¿½ï¿½ï¿½ï¿½,15000000,1.356285e+10,Comedy,ISSEI / ï¿½ï¿½ï¿½ï¿½,1788,Japan,JP,Comedy,...,900000.0,2014.0,Jul,23.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924
702,703,Bayashi TV,14900000,8.615619e+09,Entertainment,ýýýýýýýýýýýýýýýýýýBAYASHITV,93,Japan,JP,Autos,...,9.0,2012.0,Nov,19.0,63.2,126226568.0,2.29,115782416.0,36.204824,138.252924


In [40]:
# 登録者数でトップ10のチャンネルを抽出
top_10_subscribers = japan_channels.nlargest(10, 'subscribers')

# バーチャートでトップ10を表示
fig1 = px.bar(top_10_subscribers, x='subscribers', y='Youtuber', orientation='h', text='subscribers',
              color='subscribers', labels={'subscribers': '登録者数（十億単位）'},
              color_continuous_scale='Viridis')
fig1.update_traces(marker_line_color='rgb(8,48,107)', marker_line_width=1.5,
                    opacity=0.8, textposition='inside')
fig1.update_layout(title_text='登録者数による日本のYouTubeチャンネルトップ10', yaxis_title='YouTubeチャンネル',
                   xaxis_title='登録者数（十億単位）', height=500)
fig1.show()
